In [50]:
!pip install wikipedia
!pip install wptools
import pandas as pd
import spacy
import nltk 
nltk.download('punkt')
import requests
import wikipedia
import wptools
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [169]:
def scarpper(src, n): #function to get text from web
    count = 0 # counter to count that the loop runs n times only
    try:
        page = wikipedia.page(src) #wikipedia page for the item
        l = page.links #get all links from the page
        
        for link in l: # loop to extract text and make n text files from these wiki links in the said wikipedia page
            if count < n+1:
                p = wikipedia.page(title = link)
                path = '/content/random_articles'
                file_name = "{}.txt".format(p.title)
                complete_path = os.path.join(path, file_name)
                with open(complete_path, "w", encoding= 'utf-8' ) as text_file: # writing text into text files
                    text_file.write(p.summary)
                count = count +1    
            else:
                break    
            
    except Exception as e:
        print(e)

In [86]:
# first argument 'topic' , second argument 'number for relevant summaries to look for'
scarpper('topic_X',30) # executing scarpper function to fetch articles
### Topics USED IN THIS CODE: 
#baseball
#meditation
#tornadoes
#Einstein
#archers
#Algeria
#Native Mexicans
#Aztecs
#Pyramids
#Antarctic krill
#Artificial sweetener
#Jacques Cousteau
#Great Barrier Reef
#Frida Kahlo
#Mars rover
#human rights
#Classical conditioning
#Hubble Space Telescope

Page id "cleopatra iv" does not match any pages. Try another id!


In [215]:
#### reading files from folder and extracting texts
def segment_data(directory:str):
  ''' the function takes a directory in a string format as an input to read files from a directory then outputs the result in a string format
  that is compatible with spacy and nltk for further preprocessing of data.
  Then it creates a global object: 'string_text' to be used for other functions if needed.
  then the function calls for spacy and nltk to preprocess the text and segment the sentences. THE FINAL OUTPUTS IS :
  a global dataframe with four columns that contain:
  orignal_text, shared_sentences, unique_to_spacy, unique_to_nltk '''
  print('FETCHING ARTICLES FROM DIRECTORY', '*' * 60)
  files = os.scandir(directory) # creating an os object 
  content = []  
  print("\nOBJECTS : 'string_text' and 'df' ARE GLOBAL OBJECTS FOR FURTHER MANIPULATION OF YOUR CHOICE")
  for entry in files: # reading each file in the object
    if entry.is_file():
      with open(entry.path,'r') as f: # opening each file
        summaries = f.read()
        content.append(summaries) # append text data into content list
        ################################
        global string_text # making string_text global to be used in other preprocessing functions
        string_text = ' '.join(content) # converting all list elements of content object into one long string to be accepted by spacy and nltk
  print('\nDataFrame' , '*' * 60)
  nlp = spacy.load('en_core_web_sm')
  nlp.max_length = 20000000
  spacy_ = nlp(string_text)
  spacy_sents1 = [sent.text for sent in spacy_.sents] # creating list of segmented sentences of spacy
  print('\nSpacy Done With No Errors!')
  print('*'* 50)
  sentences_nltk = nltk.sent_tokenize(string_text) # segmentation nltk
  print('Nltk Done With No Errors!')
  print('\nBe careful the resulting dataframe may be unbalanced due to different results from spacy and nltk in the unique sentences:')
  global df # making the df global so we can manipulate it after if needed
  df = pd.DataFrame()
  one_spacy = pd.Series(spacy_sents1)
  one_nltk = pd.Series(sentences_nltk)
  shared = one_spacy[one_spacy.isin(one_nltk)] # filtering only shared sentences that are segmented the same by the two modules
  unique_to_spacy = one_spacy[~one_spacy.isin(one_nltk)] # filtering unique sentences segmented by spacy
  unique_to_nltk = one_nltk[~one_nltk.isin(one_spacy)] # filtering unique sentences segmented by spacy
  df['shared_sentences'] = shared
  df['unique_to_spacy'] = unique_to_spacy.reset_index(drop=True)
  df['unique_to_nltk'] = unique_to_nltk.reset_index(drop=True)
  df = pd.concat([pd.Series(string_text, name='original_text'), df], axis=1) # create new Series for original text and concatenate with DataFrame  df = df.reset_index(drop=True)
  return df

In [214]:
segment_data('./path')

FETCHING ARTICLES FROM DIRECTORY ************************************************************

OBJECTS : 'string_text' and 'df' ARE GLOBAL OBJECTS FOR FURTHER MANIPULATION OF YOUR CHOICE

DataFrame ************************************************************

Spacy Done With No Errors!
**************************************************
Nltk Done With No Errors!

Be careful the resulting dataframe may be unbalanced due to different results from spacy and nltk in the unique sentences:


,original_text,shared_sentences,unique_to_spacy,unique_to_nltk
0,Six judges of the International Criminal Court...,Six judges of the International Criminal Court...,"If the creditor breaches the accord, then the ...","If the creditor breaches the accord, then the ..."
1,NaN,The judges were elected for terms of nine year...,"This temple, built by Chola emperor Rajaraja I...","This temple, built by Chola emperor Rajaraja I..."
2,NaN,Accord and satisfaction is a contract law conc...,The Airavatesvarar temple is one among a clust...,It also reverentially displays Vaishnavism and...
3,NaN,It is one of the methods by which parties to a...,It also reverentially displays Vaishnavism and...,"It was hypothesized that certain forms, such a..."
4,NaN,The release is completed by the transfer of va...,The stone temple incorporates a chariot struct...,Though challenged in the 17th and 18th centuri...
...,...,...,...,...
1568,NaN,Voter turnout was only 25%.,NaN,NaN
1569,NaN,The 20th Century Press Archives (German: Press...,NaN,NaN
1571,NaN,It originates from the Hamburg Kolonialinstitu...,NaN,NaN
1574,NaN,In 2007 it was absorbed by the German National...,NaN,NaN


In [156]:
df.to_csv('./path') # save the dataframe into a csv